# Case Study
Create an ETL pipeline of ingestion & transform and load queries on any data set and initiate the pipeline from workflow using notebook

Part1: create a notebook with ETL queries
Part2: Run the notebook from workflow pipeline in azure Databricks workspace


In [0]:
spark.conf.set(
    "fs.azure.account.key.hexa1casestudy1adls.dfs.core.windows.net",
    "s95FXGkWbk0h1NkPgZSC9TRP8YpxQyJVh9qEeHhKpTtYUBNumLCkxDrVFu6p8KWCDwrc8fPPjB9L+ASte/d5/Q=="
)


In [0]:
dbutils.fs.ls("abfss://database@hexa1casestudy1adls.dfs.core.windows.net/")


[FileInfo(path='abfss://database@hexa1casestudy1adls.dfs.core.windows.net/Employee Sample Data 1.csv', name='Employee Sample Data 1.csv', size=148604, modificationTime=1733739857000),
 FileInfo(path='abfss://database@hexa1casestudy1adls.dfs.core.windows.net/txn.csv', name='txn.csv', size=7885867, modificationTime=1733739885000)]

In [0]:
# # Load the CSV file from ADLS
# df2 = spark.read.csv("abfss://database@hexa1casestudy1adls.dfs.core.windows.net/txn.csv", header=True, inferSchema=True)

# # Create a temporary view
# df2.createOrReplaceTempView("txn_view")

# Load the CSV file from ADLS
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://database@hexa1casestudy1adls.dfs.core.windows.net/txn.csv")

# Create a temporary view
df.createOrReplaceTempView("txn_view")

In [0]:
df.show(10)


+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            NULL|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            NULL|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            NULL|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            NULL|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7500000.0|

In [0]:
# Replace invalid characters in column names
df_renamed = df.toDF(*[col.replace(" ", "_").replace(".", "").replace(",", "") for col in df.columns])

# Save the DataFrame as a Delta table
df_renamed.write.format("delta").mode("overwrite").saveAsTable("txn_table")

df_renamed.show(10)

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account_No| TRANSACTION_DETAILS|VALUE_DATE|_WITHDRAWAL_AMT_|_DEPOSIT_AMT_|BALANCE_AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            NULL|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            NULL|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            NULL|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            NULL|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7500000.0|

In [0]:
# Replace null with 0 only in specific columns

columns_to_update = ["_WITHDRAWAL_AMT_", "_DEPOSIT_AMT_", "BALANCE_AMT"]  # Specify the columns
df_renamed = df_renamed.na.fill(0, subset=columns_to_update)

df_renamed.show(10)




+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account_No| TRANSACTION_DETAILS|VALUE_DATE|_WITHDRAWAL_AMT_|_DEPOSIT_AMT_|BALANCE_AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|             0.0|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|             0.0|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|             0.0|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|             0.0|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  7500000.0|

In [0]:
df_renamed.show(10)

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account_No| TRANSACTION_DETAILS|VALUE_DATE|_WITHDRAWAL_AMT_|_DEPOSIT_AMT_|BALANCE_AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|             0.0|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|             0.0|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|             0.0|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|             0.0|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|             0.0|     500000.0|  7500000.0|

In [0]:
%sql
SELECT * FROM txn_table
limit 50;

Account_No,TRANSACTION_DETAILS,VALUE_DATE,_WITHDRAWAL_AMT_,_DEPOSIT_AMT_,BALANCE_AMT
409000611074',TRF FROM Indiaforensic SERVICES,29-Jun-17,null,1000000.0,1000000.0
409000611074',TRF FROM Indiaforensic SERVICES,5-Jul-17,null,1000000.0,2000000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,18-Jul-17,null,500000.0,2500000.0
409000611074',TRF FRM Indiaforensic SERVICES,1-Aug-17,null,3000000.0,5500000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,16-Aug-17,null,500000.0,6000000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,16-Aug-17,null,500000.0,6500000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,16-Aug-17,null,500000.0,7000000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,16-Aug-17,null,500000.0,7500000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,16-Aug-17,null,500000.0,8000000.0
409000611074',FDRL/INTERNAL FUND TRANSFE,16-Aug-17,null,500000.0,8500000.0


Databricks visualization. Run in Databricks to view.